In [2]:
import pandas as pd
import requests
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.svm import SVC
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.cross_validation import KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.utils import shuffle
import pylab
import matplotlib.pyplot as plt
import re
import seaborn
import random
pd.set_option('display.max_columns', 500)
%matplotlib inline
import time

C:\Anaconda2\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
C:\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
df1 = pd.read_csv('https://raw.githubusercontent.com/Columbia-Intro-Data-Science/python-introduction-lee-jw/master/data_trading.csv')
slugs = df1['slug']
apikey = "INSERT API KEY" ## TrackICO Terms and conditions require apiKeys to not be shared
url = "https://api.trackico.io/v1/icos/profile/"

In [5]:
def getInfo(slug):
    proj_url = url + slug + "/?apiKey=" + apikey
    response = requests.get(proj_url).json()
    title = response["title"] #0
    slug = response["slug"] #1
    ticker = response["ticker"] #2
    ico_type = response["ico_type"] #3
    platform = response["platform"] #4
    accepted_currencies = response["finance"]["accepted_currencies"] #5
    country = response["country"] #6
    category = response["category"] #7
    rating = response["rating"] #8
    web_url = response["url"] #9
    description = response["description"] #10
    link_bitcointalk = response["links"]["bitcointalk"] #11
    pre_ico_start = response["pre_ico_start"] #12
    pre_ico_end = response["pre_ico_end"] #13
    ico_start = response["ico_start"] #14
    ico_end = response["ico_end"] #15
    token_for_sale = response["finance"]["token_for_sale"] #16
    token_supply = response["finance"]["token_supply"] #17
    soft_cap = response["finance"]["soft_cap"] #18
    soft_cap_currency = response["finance"]["soft_cap_currency"] #19
    hard_cap = response["finance"]["hard_cap"] #20
    hard_cap_currency = response["finance"]["hard_cap_currency"] #21
    token_price_in_usd = response["finance"]["token_price_in_usd"] #22
    total_raised_in_usd = response["finance"]["total_raised_in_usd"] #23
    bonus_available = response["finance"]["bonus_available"] #24
    whitelist_required = response["finance"]["whitelist_required"] #25
    kyc_required = response["finance"]["kyc_required"] #26
    restricted_countries = response["finance"]["restricted_countries"] #27
    alexa_global_rank = response["interest"]["alexa_global_rank"] #28
    twitter_participants = response["interest"]["twitter_participants"] #29
    telegram_members = response["interest"]["telegram_members"] #30
    video_views = response["interest"]["video_views"] #31
    return(title, slug, ticker, ico_type, platform, accepted_currencies, country, category, rating, web_url, description, link_bitcointalk, pre_ico_start, pre_ico_end, ico_start, ico_end, token_for_sale, token_supply, soft_cap, soft_cap_currency, hard_cap, hard_cap_currency, token_price_in_usd, total_raised_in_usd, bonus_available, whitelist_required, kyc_required, restricted_countries, alexa_global_rank, twitter_participants, telegram_members, video_views)

In [6]:
# Testing code - append data to dataframe
df1 = pd.DataFrame(columns = list(['title','slug']))
data = getInfo('red-pulse')
df1 = df1.append({'title':data[0],'slug':data[1]}, ignore_index=True)
df1.head()

,title,slug
0,Red Pulse,red-pulse


In [7]:
# Testing code - timer
test = slugs[0:10]
print(test)
for t in test:
    df1.append({'title':t,'slug':t}, ignore_index=True)
    time.sleep(.5)

0    vice-industry-token
1                   datx
2         sentinel-chain
3            ip-exchange
4               effectai
5                friendz
6               leadcoin
7                 helbiz
8                   jet8
9                 dockio
Name: slug, dtype: object


In [8]:
# Send request every 3 seconds
df = pd.DataFrame(columns = list(['title','slug','ticker','ico_type','platform','accepted_currencies','country','category','token_for_sale','token_supply','soft_cap','soft_cap_currency','hard_cap','hard_cap_currency','token_price_in_usd','total_raised_in_usd','bonus_available','whitelist_required','kyc_required','restricted_countries','alexa_global_rank','twitter_participants','telegram_members','video_views']))
df.head()

counter = 0
for slug in slugs:
    data = getInfo(slug)
    df = df.append({'title': data[0], 'slug': data[1], 'ticker': data[2], 'ico_type': data[3], 'platform': data[4], 'accepted_currencies': data[5], 'country': data[6], 'category': data[7], 'rating': data[8], 'token_for_sale': data[16], 'token_supply': data[17], 'soft_cap': data[18], 'soft_cap_currency': data[19], 'hard_cap': data[20], 'hard_cap_currency': data[21], 'token_price_in_usd': data[22], 'total_raised_in_usd': data[23], 'bonus_available': data[24], 'whitelist_required': data[25], 'kyc_required': data[26], 'restricted_countries': data[27], 'alexa_global_rank': data[28], 'twitter_participants': data[29], 'telegram_members': data[30], 'video_views': data[31]}, ignore_index=True)
    counter += 1
    print(counter)
    time.sleep(3)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [9]:
df.head()

,title,slug,ticker,ico_type,platform,accepted_currencies,country,category,token_for_sale,token_supply,soft_cap,soft_cap_currency,hard_cap,hard_cap_currency,token_price_in_usd,total_raised_in_usd,bonus_available,whitelist_required,kyc_required,restricted_countries,alexa_global_rank,twitter_participants,telegram_members,video_views,rating
0,Vice Industry Token,vice-industry-token,VIT,Trading,Ethereum,[],Canada,Games & Entertainment,2000000000,4000000000,None,None,None,None,NaN,16156000,False,None,None,[],144163,11097,10502,None,3.3
1,DATx,datx,DATX,Trading,Ethereum,[],Singapore,Platform,None,None,None,None,None,None,NaN,0,False,None,None,[],166222,47393,27854,None,0.0
2,Sentinel Chain,sentinel-chain,SENC,Trading,Ethereum,[ETH],Singapore,Finance,200000000,500000000,None,None,None,None,NaN,14400000,False,None,True,"[China, United States of America]",188245,3924,10255,15443,4.4
3,IP Exchange,ip-exchange,IPSX,Trading,Ethereum,[ETH],Switzerland,Platform,303529411,1800000000,None,None,None,None,0.0122,7904000,False,True,True,"[Canada, China, United States of America]",334661,2761,5873,4689,4.6
4,EFFECT.AI,effectai,AIX,Trading,NEO,"[NEO, GAS]",Netherlands,Platform,260000000,650000000,None,None,None,None,None,13246927,True,True,True,"[China, United States of America]",133342,10292,23168,None,4.6


In [10]:
df.to_csv('data_final.csv')